In [60]:
import pandas as pd
import numpy as np

np.float_ = np.float64

from prophet import Prophet
import requests
from datetime import datetime, timedelta

m = Prophet()

In [61]:
# Get data

end_time = datetime.now()
print(end_time.timestamp())
start_time = (end_time - timedelta(seconds=3600.0)).timestamp()
print(start_time)

transaction_types = ["CONTRACTCALL"]

from typing import Union
from pprint import pprint
def get_data(start_time: float | int, limit: int, transaction_type: str, order: Union["asc", "desc"]):
    url = f"https://testnet.mirrornode.hedera.com/api/v1/transactions?limit={limit}&order={order}&timestamp=gte%3A{start_time}&transactiontype={transaction_type}"
    data = requests.get(url).json()
    return data["transactions"]

transactions = get_data(start_time, 100, transaction_types[0], "asc")

1723893051.708523
1723889451.708523


In [62]:
import json

df = pd.read_json(json.dumps(transactions))
df.tail()

/tmp/ipykernel_6918/1234645194.py:3: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json.dumps(transactions))


,bytes,charged_tx_fee,consensus_timestamp,entity_id,max_fee,memo_base64,name,nft_transfers,node,nonce,parent_consensus_timestamp,result,scheduled,staking_reward_transfers,token_transfers,transaction_hash,transaction_id,transfers,valid_duration_seconds,valid_start_timestamp
26,NaN,158400000,1.723893e+09,0.0.4672711,200000000,,CONTRACTCALL,[],0.0.5,0,NaN,SUCCESS,False,[],[],jeDAXLV2XSqVSdxN3OYdSgmlSu79Vbih4DNuYrjiHUgXOc...,0.0.2119-1723892758-084217830,"[{'account': '0.0.98', 'amount': 158400000, 'i...",120.0,1.723893e+09
27,NaN,0,1.723893e+09,0.0.359,0,,CONTRACTCALL,[],None,1,1.723893e+09,SUCCESS,False,[],[],of1R+b3EAnu/RFi8Y2gsl4DbUml/M59P74psGCRxQuv80l...,0.0.2119-1723892758-084217830,[],NaN,1.723893e+09
28,NaN,0,1.723893e+09,0.0.359,0,,CONTRACTCALL,[],None,2,1.723893e+09,SUCCESS,False,[],[],N9MdDVJ2dbCN8BGW1JH4VDniPpJHrolisfCX+D7eD46QgO...,0.0.2119-1723892758-084217830,[],NaN,1.723893e+09
29,NaN,0,1.723893e+09,0.0.359,0,,CONTRACTCALL,[],None,3,1.723893e+09,SUCCESS,False,[],[],FdHA1ATP9ysZeTTVeiPWhmccY6GC9gnC+OKSfdAi0ASvvv...,0.0.2119-1723892758-084217830,[],NaN,1.723893e+09
30,NaN,0,1.723893e+09,0.0.359,0,,CONTRACTCALL,[],None,5,1.723893e+09,SUCCESS,False,[],[],pWVARPsNIg1ylL3TV2agf1dMDv4NgKDujWNuU/S4bR8/y8...,0.0.2119-1723892758-084217830,[],NaN,1.723893e+09


In [63]:
new_df = df.drop(["bytes", "entity_id", "max_fee", "memo_base64", "name", "node", "nonce", "parent_consensus_timestamp", "consensus_timestamp", "transaction_hash", "valid_duration_seconds", "valid_start_timestamp", "nft_transfers", "token_transfers", "staking_reward_transfers"], axis="columns")
new_df.tail()

,charged_tx_fee,result,scheduled,transaction_id,transfers
26,158400000,SUCCESS,False,0.0.2119-1723892758-084217830,"[{'account': '0.0.98', 'amount': 158400000, 'i..."
27,0,SUCCESS,False,0.0.2119-1723892758-084217830,[]
28,0,SUCCESS,False,0.0.2119-1723892758-084217830,[]
29,0,SUCCESS,False,0.0.2119-1723892758-084217830,[]
30,0,SUCCESS,False,0.0.2119-1723892758-084217830,[]


In [71]:
new_df.head()

,charged_tx_fee,result,scheduled,transaction_id,transfers
0,0,REVERTED_SUCCESS,False,0.0.902-1723889449-455599807,[]
1,0,REVERTED_SUCCESS,False,0.0.902-1723889449-455599807,[]
2,0,REVERTED_SUCCESS,False,0.0.902-1723889449-455599807,[]
3,0,REVERTED_SUCCESS,False,0.0.902-1723889516-033567002,[]
4,0,REVERTED_SUCCESS,False,0.0.902-1723890066-690469977,[]


In [70]:
pd.json_normalize(new_df, record_path=[4], meta=["charged_tx_fee", "result", "scheduled", "transaction_id", "transfers"])

TypeError: charged_tx_fee has non list value g for path 4. Must be list or null.